In [6]:
import os
from glob import glob
import sys
import pandas as pd
import MDAnalysis as mda
import warnings
import re
from tqdm import tqdm
from multiprocessing import Pool
import gc

os.chdir("/home/adri/Projects/phd/pipeline")

sys.path.append("src")
from utils import identify_ligands

In [7]:
# create processed contacts folder
os.makedirs('data/interim/processed_contacts', exist_ok=True)
# for file in contacts folder
def process_contacts(contacts_file):
    # extract system and replica from file name
    system = contacts_file.split("/")[-2]
    replica = contacts_file.split("/")[-1].split(".")[0]
    # get the ligand resid or chain id
    system_raw_path = f"data/raw/simulations/{system}"
    top_list = glob(f"{system_raw_path}/*.psf") + glob(f"{system_raw_path}/*.top") + glob(f"{system_raw_path}/*.pdb")
    assert top_list, f"No topology file found for {system}"
    top = top_list[0]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=UserWarning)
        u = mda.Universe(top)
        ligands = identify_ligands(u)
        del u
    # check that there is only one ligand
    assert len(ligands) == 1, f"More than one (or none) ligands found in {system}"
    ligand = list(ligands)[0]

    if "SEG" in ligand:
        # if ligand is peptide replace <CHAIN>:<RESNAME> with <CHAIN>:LIG
        chain = ligand.split(":")[1][0]
        string_to_replace = fr"{chain}:[A-Z]{{3}}:[0-9]+"
        replacement_string = fr"{chain}:LIG:1"
    else:
        # if ligand is small molecule replace resname with LIG
        string_to_replace = f"{ligand}:[0-9]+"
        replacement_string = "LIG:1"

    output_folder = f"data/interim/processed_contacts/{system}"
    os.makedirs(output_folder, exist_ok=True)
    output_path = f"{output_folder}/{replica}.tsv"
    # read contacts file
    with open(contacts_file, "r") as contact_file:
        with open(output_path, 'w') as output_file:
            # read file
            for og_line in contact_file:
                # replace ligand name
                line = re.sub(string_to_replace, replacement_string, og_line)
                # write line to output file
                output_file.write(line)
        
    gc.collect()



In [8]:
with Pool(15) as p:
    p.map(process_contacts, glob("data/interim/contacts/*/*.tsv"))